In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

load_dotenv(override=True)

True

In [2]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content

    return response

In [3]:
client = OpenAI(
    # api_key=os.getenv("RUNPOD_TOKEN"),
    # base_url=os.getenv("RUNPOD_CHATBOT_URL"),
    api_key=os.environ.get("OPENAI_API_KEY")
)
# model_name = os.getenv("MODEL_NAME")
model_name = os.getenv("OPENAI_MODEL_NAME")

### Get LLM response

In [ ]:
response = get_chatbot_response(client, model_name, [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
], temperature=0)

### Prompt Engineering

#### Structured output

In [ ]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    country: the country that you will get the capital of 
    capital: the capital of the country stated
}
]
"""

In [ ]:
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':"What's the capital of Italy?"})
response = get_chatbot_response(client,model_name,messages)

In [ ]:
print(response)

In [ ]:
json_response = json.loads(response)
json_response

### input structuring

In [ ]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
``
"""

In [ ]:
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
response = get_chatbot_response(client,model_name,messages)

In [ ]:
print(response)

In [ ]:
json_response = json.loads(response)
json_response

### Give the model time to think (Chain of thought)

- https://arxiv.org/pdf/2205.11916

In [ ]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [ ]:
print(response)

In [ ]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [ ]:
print(response)

In [ ]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [ ]:
print(response)

### RAG - Retrieval Augmented Generation

#### Asking about a subject that the LLM does not know anything about

In [ ]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [ ]:
print(response)

#### Giving Context to the unknown subject

In [5]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [ ]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [ ]:
print(response)

#### Automatically extract context data from database

In [4]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [6]:
data = [
    iphone_16,
    samsung_s23,
]
user_prompt = """What's new in iphone 16?"""


In [7]:
embedding_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [8]:
def get_embedding(embedding_client,model_name,text_input):
    output = embedding_client.embeddings.create(input = text_input,model=model_name)
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings

In [9]:
user_prompt_embeddings = get_embedding(
    embedding_client,
    os.getenv("OPENAI_EMBEDDING_MODEL_NAME"),
    user_prompt
)

In [10]:
user_prompt_embeddings

[[0.033275119960308075,
  0.019743597134947777,
  -0.014908982440829277,
  0.016448495909571648,
  -0.017312785610556602,
  -0.004331572446972132,
  0.014611883088946342,
  0.022228429093956947,
  -0.009365379810333252,
  -0.044051721692085266,
  -0.012451160699129105,
  -0.011890723370015621,
  -0.08334985375404358,
  -0.015692243352532387,
  -0.009702992625534534,
  -0.016313452273607254,
  -0.059311822056770325,
  -0.016097379848361015,
  -0.021647734567523003,
  0.0003688420110847801,
  0.014463333413004875,
  0.02129661664366722,
  -0.002937231445685029,
  -0.008723915554583073,
  0.03262690454721451,
  -0.00427080225199461,
  -0.028818629682064056,
  -0.04297136142849922,
  0.013355962932109833,
  0.007170896511524916,
  0.008845455944538116,
  -0.040054384618997574,
  0.0695212334394455,
  0.006762384902685881,
  -0.017717920243740082,
  -0.020675409585237503,
  0.018231092020869255,
  -0.02776527777314186,
  0.015435658395290375,
  0.04094568267464638,
  -0.023416824638843536,


In [14]:
user_prompt_embedding = user_prompt_embeddings[0]

In [ ]:
data_embeddings = [get_embedding(
    embedding_client,
    os.getenv("OPENAI_EMBEDDING_MODEL_NAME"),
    x
)[0] for x in data]


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
data_similaraty_scores = cosine_similarity(user_prompt_embeddings, data_embeddings)

In [28]:
data_similaraty_scores

array([[0.74707574, 0.38846034]])

In [ ]:
closest_entry_index=data_similaraty_scores.argmax() # 最大值的索引
closest_entry_index

0

In [30]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [31]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
""" 

In [32]:
messages = [{'role':'user','content':user_prompt_with_data}]
response = get_chatbot_response(client,model_name,messages)
print(response)

1. Larger displays: The base model iPhone 16 features a 6.1-inch display, while the iPhone 16 Plus has a 6.7-inch screen. The Pro models have even larger displays, measuring 6.3 and 6.9 inches.

2. A18 chip: The iPhone 16 is powered by the new A18 chip (A18 Pro for the Pro models), offering significant performance improvements and enhanced neural engine capabilities.

3. Upgraded camera systems: The base iPhone 16 has a dual-camera setup with a 48MP main sensor, while the Pro models offer a 48MP Ultra Wide and 5x telephoto camera.

4. Advanced audio features: The iPhone 16 introduces features like "Audio Mix," which uses machine learning to separate background sounds from speech for better audio capture during video recording.

5. Longer battery life: The iPhone 16 Pro Max is claimed to have the longest-lasting battery of any iPhone, with extended battery life compared to previous models.

6. USB-C support: Apple has switched to USB-C for faster charging and data transfer, and the Pro 